In [67]:
import pandas as pd
import numpy as np

df=pd.read_csv('Dataset-SA.csv')


In [68]:
df.head

<bound method NDFrame.head of                                              product_name product_price Rate  \
0       Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    5   
1       Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    5   
2       Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    3   
3       Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    1   
4       Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    3   
...                                                   ...           ...  ...   
205047  cello Pack of 18 Opalware Cello Dazzle Lush Fi...          1299    5   
205048  cello Pack of 18 Opalware Cello Dazzle Lush Fi...          1299    5   
205049  cello Pack of 18 Opalware Cello Dazzle Lush Fi...          1299    3   
205050  cello Pack of 18 Opalware Cello Dazzle Lush Fi...          1299    5   
205051  cello Pack of 18 Opalware Cello Dazzle Lush Fi...          1299    4   

         

In [69]:
df.shape

(205052, 6)

In [70]:
df.describe()

,product_name,product_price,Rate,Review,Summary,Sentiment
count,205052,205052,205052,180388,205041,205052
unique,958,525,8,1324,92923,3
top,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,wonderful,good,positive
freq,6005,9150,118765,9016,17430,166581


In [71]:
df.isnull().sum()

product_name         0
product_price        0
Rate                 0
Review           24664
Summary             11
Sentiment            0
dtype: int64

In [72]:
df = df.dropna(subset=['Review', 'Summary'])
print(df.shape) 

(180379, 6)


In [73]:
df.isnull().sum()

product_name     0
product_price    0
Rate             0
Review           0
Summary          0
Sentiment        0
dtype: int64

In [74]:
# Example with pandas
df_positive = df[df['Sentiment'] == 'positive'].sample(n=8807, random_state=42)
df_negative = df[df['Sentiment'] == 'negative'].sample(n=8807, random_state=42)
df_neutral  = df[df['Sentiment'] == 'neutral']   # keep all

df_balanced = pd.concat([df_positive, df_negative, df_neutral])
print(df_balanced['Sentiment'].value_counts())


Sentiment
positive    8807
negative    8807
neutral     8807
Name: count, dtype: int64


In [75]:
df_balanced.shape

(26421, 6)

In [76]:
print(df_balanced['Review'].head())

22276     mind-blowing purchase
184721           simply awesome
18188     slightly disappointed
179306                  awesome
69723                   awesome
Name: Review, dtype: object


In [58]:
pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl (273 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sowmi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [81]:
from sklearn.preprocessing import LabelEncoder

def preprocess_text(text):
    text = text.lower()
    
    # Remove punctuation
    text = ''.join(char for char in text if char.isalnum() or char.isspace())
    
    # Tokenize and remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    # Generate bigrams (n-grams with 2 words)
    bigrams = [' '.join(tokens[i:i+2]) for i in range(len(tokens)-1)]
    tokens += bigrams
    # Join back into string
    return ' '.join(tokens)

df_balanced['clean_review'] = df_balanced['Review'].apply(preprocess_text)


X = df_balanced['clean_review']
y = df_balanced['Sentiment']

le = LabelEncoder()
y = le.fit_transform(y)

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42, stratify=y)

In [82]:
from collections import Counter

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

print("Train label counts:", Counter(y_train))
print("Test label counts:", Counter(y_test))

Train shape: (21136, 372)
Test shape: (5285, 372)
Train label counts: Counter({np.int64(0): 7046, np.int64(2): 7045, np.int64(1): 7045})
Test label counts: Counter({np.int64(2): 1762, np.int64(1): 1762, np.int64(0): 1761})


In [83]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import f1_score
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))
print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.6862819299905393
F1 Score: 0.6765165688278928
              precision    recall  f1-score   support

    negative       0.79      0.77      0.78      1761
     neutral       0.57      0.44      0.50      1762
    positive       0.68      0.85      0.75      1762

    accuracy                           0.69      5285
   macro avg       0.68      0.69      0.68      5285
weighted avg       0.68      0.69      0.68      5285

